In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
GEN_BACK_TR = True

In [2]:
#python basics
from matplotlib import pyplot as plt
import math, os, re, time
import numpy as np, pandas as pd, seaborn as sns
from multiprocessing import Pool
#nlp augmentation
!pip install --quiet googletrans==3.1.0a0
!pip install --quiet textattack
from googletrans import Translator

#model evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold

#for fast parallel processing
from dask import bag, diagnostics

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 50.7 MB/s eta 0:00

In [3]:
combined_dataset=pd.read_csv("/content/drive/MyDrive/NLP_Mini_Project/Data_Augmentation_Techniques/Bert_Augmentation/Train_clean.csv")

In [27]:
from textattack.augmentation import EmbeddingAugmenter
from textattack.augmentation import WordNetAugmenter
embed_aug = EmbeddingAugmenter()
wordnet_aug = WordNetAugmenter()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [26]:
def back_translate(sequence,label):
    languages = ['en']
    #instantiate translator
    translator = Translator()
    output_sequence = [sequence]
    label_sequence = [label]
    #store original language so we can convert back
    org_lang = 'mr'
    #randomly choose language to translate sequence to  
    for language  in languages:
      #translate to new language and back to original
      translated = translator.translate(sequence, dest = language).text
      
      translated_back1 = translator.translate(wordnet_aug.augment(translated)[0], dest = org_lang).text
      translated_back2 = translator.translate(embed_aug.augment(translated)[0], dest = org_lang).text
      output_sequence.append(translated_back1)
      label_sequence.append(label)
      output_sequence.append(translated_back2)
      label_sequence.append(label)
    
    news_list.extend(output_sequence)
    label_list.extend(label_sequence)


In [31]:
def back_translate_parallel(dataset):
    news = (dataset['headline']).tolist()
    labels = (dataset['label']).tolist()
    with diagnostics.ProgressBar():
        for i in range(0,len(news)):
          print(i)
          back_translate(news[i],labels[i])

In [32]:
news_list = list()
label_list=list()
if True:
#now we apply translation augmentation
    back_translate_parallel(combined_dataset)
    dic = {"news":news_list,"label":label_list}
    df = pd.DataFrame(dic)
    df.to_csv("Augmented_Train.csv",index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
news_list

['सचिन तेंडुलकरने बाळासाहेबांच्या आठवणींना दिला उजाळा',
 'Sachin Tendulkar brought light to the store of Balasaheb',
 'Sachin Tendulkar brought light to the recollections of Balasaheb',
 'बहुजन वंचित आघाडीची पहिली यादी जाहीर प्रकाश आंबेडकरांच्या मतदारसंघावरुन सस्पेंन्स कायम',
 'बहुजन वंचित आघाडीची पहिली यादी जाहीर. प्रकाश आंबेडकर यांच्या मतदारसंघावरून सस्पेन्स कायम आहे',
 'बहुजन वंचित आघाडीची पहिली यादी जाहीर. प्रकाश आंबेडकर यांच्या मतदारसंघातूनही नाराजी कायम आहे',
 'गृहमंत्रिपद हे पार्ट टाइम नाही राजीनामा द्या तृप्ती देसाईंची मुख्यमंत्र्यांवर टीका',
 'गृहमंत्रीपद अर्धवेळ नाही, तृप्ती देसाईंची फोरमॅनवर टीका',
 'गृहमंत्रीपद अर्धवेळ नाही, राजीनामा द्या तृप्ती देसाईंची मुख्यमंत्र्यांवर टीका',
 'पुण्यात आघाडीचा पोपट पुन्हा जिवंत ',
 'पुण्यात आघाडीचा पोपट पुन्हा सावध झाला आहे',
 'आघाडीचा पोपट पुण्यात पुन्हा जिवंत झाला आहे',
 'शरद पवार यांनी मिलिंद नार्वेकर यांच्या कानात काय सांगितलं असेल',
 'मिलिंद नार्वेकरांच्या कानात शरद पवार काय म्हणाले?',
 'मिलिंद नार्वेकरांच्या वाट्याला शरद पवार कुठे म